In [5]:
import matplotlib.pyplot as plt
import numpy as np

from vessel_models import VesselModel
from controller import MPC, MHE

In [6]:
num_trajectories = 50
for iter_n in range(47, num_trajectories):
    print(f"Running trial {iter_n}")
    # Global simulation variables (used by both MPC and MHE)
    N = 100
    dt = 0.1
    ns = 6
    nu = 3

    # MHE simulation variables
    nc = 16 + 6 # hydrodynamic coefficients + 6 tweak factors
    N_mhe = 10

    # MPC simulation variables
    N_mpc = 10 # default 10
    N_sim = N + N_mpc

    # Cost matrices
    Qvec = 3*np.ones(ns)
    Qvec[3:] = [0, 0, 0]

    Rvec = np.ones(nu)
    Qfvec = 10*np.ones(ns)
    Pvec = 0.3*np.ones(nc)

    # Reference trajectory and inputs
    Sload = np.loadtxt(f"trajectories/states/dircol_soln_{iter_n}.csv", delimiter=',')
    Uload = np.loadtxt(f"trajectories/inputs/dircol_inpu_{iter_n}.csv", delimiter=',')

    # MPC reference trajectory augmentation
    Sf = Sload[-1]
    Sref = np.tile(Sf, (N_sim, 1))
    Sref[:N] = Sload

    # Obstacle States
    #num_obstacles = 2
    #obstacles = np.zeros((num_obstacles,3))
    #obstacles[0] = [0.25, 0.25, 0.1]
    #obstacles[1] = [0.75, 0.75, 0.1]
    obstacles = np.loadtxt(f"trajectories/obstacles/dircol_obst_{iter_n}.csv", delimiter=',')

    # Vessel Models
    model = VesselModel()
    model_unc = VesselModel(mismatch=True)

    # --- SOLVER BEGINS HERE --- #
    trial_version = "mismatch_vel"
    using_MHE = True

    U_soln = np.zeros((N-1, nu))
    S_soln = np.zeros((N, ns))
    S_soln[0] = Sref[0]

    c_init = np.zeros(nc)
    c_init[-9:-6] = [23.8, 1.76, 0.046] # warm start with measured mass values
    C_soln = np.tile(c_init, (N-1, 1))
    #C_soln = np.zeros((N-1, nc))

    Z_soln = np.zeros((N-1, N_mpc*ns + (N_mpc-1)*nu))

    MHE_coeff_test = np.zeros(nc)
    MHE_coeff_test[:13] = [-0.7225, -1.3274, -0.8612, -36.2823, 0.1079, 0.1052, -0.5,
                        -1.0, -2.0, -10.0, 0.0, 0.0, -1.0]

    mhe_traj_init = np.zeros((N_mhe, ns))
    mhe_traj_init[0] = [0, 0, np.pi/4, 0, 0, 0]
    mhe_inpu_init = np.tile([10, 0, 2], (N_mhe-1, 1))
    mhe_C_init = C_soln[0]
    #mhe_C_init = np.zeros((1, nc))

    for i in range(N_mhe-1):
        Sn = mhe_traj_init[i]
        Un = mhe_inpu_init[i]
        mhe_traj_init[i+1] = model.rk4(Sn, Un, dt)

    mhe_optimizer = MHE(model, N_mhe, ns, nu, nc, dt, Qvec, Rvec, Qfvec, Pvec,
                    mhe_C_init, mhe_traj_init, mhe_inpu_init)
    mhe_optimizer.exec_MPC()
    mhe_C_init = mhe_optimizer.sol.x
    C_soln[0] = mhe_C_init

    for i in range(N-1):
        s0 = S_soln[i]
        sf = Sref[i+N_mpc-1] # use for solving following a trajectory
        sref = Sref[i:i+N_mpc]
        trajoptimizer = MPC(model, N_mpc, ns, nu, dt, Qvec, Rvec, Qfvec, s0, sf, sref, obstacles)
        
        if not using_MHE:
            trajoptimizer.exec_MPC(version="with_traj")

        elif i >= N_mhe:
            sref_mhe = S_soln[i-N_mhe:i]
            uref_mhe = U_soln[i-N_mhe:i-1]
            c0 = C_soln[i-N_mpc]
            mhe_optimizer = MHE(model, N_mhe, ns, nu, nc, dt, Qvec, Rvec, Qfvec, Pvec, c0, sref_mhe, uref_mhe)
            mhe_optimizer.exec_MPC()
            mhe_coeff = mhe_optimizer.sol.x
            C_soln[i-N_mpc+1] = mhe_coeff

            trajoptimizer.exec_MPC(version="with_MHE", MHE_coeff=mhe_coeff)

        else:
            trajoptimizer.exec_MPC(version="with_MHE", MHE_coeff=mhe_C_init)

        Z_soln[i] = trajoptimizer.sol.x
        S, U = trajoptimizer.flat2vec(trajoptimizer.sol.x)
        U_soln[i] = U[0]
        
        if trial_version == "base": S_soln[i+1] = model.rk4(s0, U[0], dt)
        elif trial_version == "base_vel": S_soln[i+1] = model.rk4_addvel(s0, U[0], dt) # try this with a bit of mismatch as well to see if MHE performs better than MPC
        elif trial_version == "mismatch": S_soln[i+1] = model_unc.rk4(s0, U[0], dt)
        elif trial_version == "mismatch_vel": S_soln[i+1] = model_unc.rk4_addvel(s0, U[0], dt) # try this with a bit of mismatch as well to see if MHE performs better than MPC
    
    # --- PLOTTING AND DATA SAVING STARTS HERE --- #
    np.savetxt(f"solves_MHEMPC/states/{trial_version}/mhempc_soln_{iter_n}.csv", S_soln, delimiter=",")
    np.savetxt(f"solves_MHEMPC/inputs/{trial_version}/mhempc_inpu_{iter_n}.csv", U_soln, delimiter=",")
    np.savetxt(f"solves_MHEMPC/planned_raw/{trial_version}/mhempc_planned_{iter_n}.csv", Z_soln, delimiter=",")

    ps = S_soln.T
    ps_ref = Sref.T
    fig, axes = plt.subplots(1, 1, figsize=(20, 20))

    for obstacle in obstacles:
        obs_x, obs_y, obs_r = obstacle
        circle = plt.Circle((obs_x, obs_y), obs_r, color='r')
        axes.add_patch(circle)

    for i in range(len(ps[1])):
        dx = 0.05*np.cos(ps[2,i])
        dy = 0.05*np.sin(ps[2,i])
        axes.arrow(ps[0,i], ps[1, i], dx, dy, head_width=0.025, head_length=0.025, fc='green', ec='green')
        
    axes.scatter(ps_ref[0], ps_ref[1])
    axes.scatter(ps[0], ps[1])
    axes.set_aspect('equal', adjustable='box')

    plt.savefig(f"solves_MHEMPC/figures/{trial_version}/mhempc_figu_{iter_n}.png")
    plt.close()
    #plt.show()

Running trial 47
{'X_u': -0.44500000000000006, 'X_uu': -1.6547999999999998, 'Y_v': -0.7223999999999999, 'Y_vv': -71.5646, 'Y_r': 1.2158, 'N_v': 1.2104, 'N_r': 0.0, 'N_rr': -1.0, 'X_du': -3.0, 'Y_dv': -19.0, 'Y_dr': 1.0, 'N_dv': 1.0, 'N_dr': -1.0, 'm': 29.75, 'I_z': 2.2, 'x_g': 0.057499999999999996}


AssertionError: NaN or Inf in constraints